# Imports

In [3]:
%pip install pandas numpy matplotlib seaborn tqdm

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.5-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached matplotlib-3.10.1-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.2-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.57.0-cp313-cp313-win_amd64.whl.metadata (104 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-win_amd64.whl.metadata (6.3 kB)
  Using cached pillow-11.2.1-cp313-cp313-win_amd64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   -------------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python313\\site-packages\\pytz\\zoneinfo\\America\\North_Dakota\\Beulah'
Check the permissions.



In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
from collections import defaultdict
from tqdm import tqdm
import json

# Load Data

In [5]:
import pandas as pd

df = pd.read_csv('../dataset/weighted_score_above_08.csv')

df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15232\972847953.py:3: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../dataset/weighted_score_above_08.csv')


,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
0,147449116,10,Counter-Strike,76561199183984450,51,12,2548,0,2480,1696305457,...,1,99,2,0.889438,0,1,0,0,1,NaN
1,147374264,10,Counter-Strike,76561198099573060,226,13,2369,0,2361,1696096555,...,1,122,8,0.914834,0,1,0,0,1,NaN
2,147357703,10,Counter-Strike,76561199080026894,118,23,13501,212,12957,1697630734,...,1,599,20,0.968375,3,1,0,0,1,NaN
3,147345102,10,Counter-Strike,76561198068970227,28,1,10668,640,9906,1698261011,...,1,59,12,0.826206,0,0,0,0,1,NaN
4,147284743,10,Counter-Strike,76561199137893460,19,5,543,10,526,1697810991,...,1,128,24,0.853612,0,1,0,0,1,NaN


In [7]:
with open('../saeed/combined_df.json', 'r') as f:
    sample_data = [json.loads(line) for line in f]

In [8]:
# Extract valid appids from the sample file
valid_appids = {entry['appid'] for entry in sample_data}

# Filter df based on appids in the sample
df = df[df['appid'].isin(valid_appids)]

# Continue processing
df_games = df[['appid', 'game']]
df_games = df_games.drop_duplicates(subset='appid')

appid2game = df_games.set_index('appid')['game'].to_dict()
game2appid = df_games.set_index('game')['appid'].to_dict()

del df_games

df = df[['appid', 'author_steamid', 'voted_up']]

In [9]:
appid_to_genres = {entry['appid']: entry['genre'] for entry in sample_data}

# Model Experiments

In [10]:
df.columns

Index(['appid', 'author_steamid', 'voted_up'], dtype='object')

## Non-Personalized Recommendations
This section implements methods for non-personalized recommendations, including weighted scoring, average rating, and Bayesian scoring.

In [11]:
# Calculate average rating for each appid
average_ratings = df.groupby('appid')['voted_up'].mean()
average_ratings = average_ratings.sort_values(ascending=False)

average_ratings.head()

appid
2608590    1.0
10         1.0
2607350    1.0
30         1.0
2599300    1.0
Name: voted_up, dtype: float64

In [12]:
# Weighted scoring: consider both average rating and number of votes
vote_counts = df.groupby('appid')['voted_up'].count()
weighted_scores = (average_ratings * vote_counts) / (vote_counts + 10)
weighted_scores = weighted_scores.sort_values(ascending=False)

weighted_scores.head()

appid
413150    0.993778
550       0.992418
264710    0.991809
294100    0.991564
4000      0.990450
Name: voted_up, dtype: float64

In [13]:
# Bayesian scoring: incorporate prior knowledge
C = df['voted_up'].mean()  # Global average
m = 10  # Minimum votes required to consider

bayesian_scores = (vote_counts * average_ratings + m * C) / (vote_counts + m)
bayesian_scores = bayesian_scores.sort_values(ascending=False)

bayesian_scores.head()

appid
550       0.998820
644560    0.998336
264710    0.998097
294100    0.998040
48700     0.997774
Name: voted_up, dtype: float64

In [14]:
# Recommend top 5 games based on average rating
top_average_rating = average_ratings.head(5).index
recommended_games_avg = [appid2game[appid] for appid in top_average_rating]

print('Top 5 games by average rating:', recommended_games_avg)

Top 5 games by average rating: ['Astrea Official Artbook', 'Counter-Strike', 'VPet - PCat', 'Day of Defeat', 'Dead Signal']


In [15]:
# Recommend top 5 games based on weighted scoring
top_weighted_scores = weighted_scores.head(5).index
recommended_games_weighted = [appid2game[appid] for appid in top_weighted_scores]

print('Top 5 games by weighted scoring:', recommended_games_weighted)

Top 5 games by weighted scoring: ['Stardew Valley', 'Left 4 Dead 2', 'Subnautica', 'RimWorld', "Garry's Mod"]


In [16]:
# Recommend top 5 games based on Bayesian scoring
top_bayesian_scores = bayesian_scores.head(5).index
recommended_games_bayesian = [appid2game[appid] for appid in top_bayesian_scores]

print('Top 5 games by Bayesian scoring:', recommended_games_bayesian)

Top 5 games by Bayesian scoring: ['Left 4 Dead 2', 'Mirror', 'Subnautica', 'RimWorld', 'Mount & Blade: Warband']


## Evaluating Recommendation Performance
This section evaluates the performance of the recommendations using precision, recall, and F1-score.

In [17]:
# Define a ground truth for evaluation (e.g., top-rated games by users)
ground_truth = set(df[df['voted_up'] == 1]['appid'].value_counts().head(10).index)

# Helper function to calculate precision, recall, and F1-score
def evaluate_recommendations(recommended, ground_truth):
    recommended_set = set(recommended)
    true_positives = len(recommended_set & ground_truth)
    precision = true_positives / len(recommended_set) if recommended_set else 0
    recall = true_positives / len(ground_truth) if ground_truth else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1_score

# Evaluate average rating recommendations
precision_avg, recall_avg, f1_avg = evaluate_recommendations(top_average_rating, ground_truth)
print(f'Average Rating - Precision: {precision_avg:.2f}, Recall: {recall_avg:.2f}, F1-Score: {f1_avg:.2f}')

# Evaluate weighted scoring recommendations
precision_weighted, recall_weighted, f1_weighted = evaluate_recommendations(top_weighted_scores, ground_truth)
print(f'Weighted Scoring - Precision: {precision_weighted:.2f}, Recall: {recall_weighted:.2f}, F1-Score: {f1_weighted:.2f}')

# Evaluate Bayesian scoring recommendations
precision_bayesian, recall_bayesian, f1_bayesian = evaluate_recommendations(top_bayesian_scores, ground_truth)
print(f'Bayesian Scoring - Precision: {precision_bayesian:.2f}, Recall: {recall_bayesian:.2f}, F1-Score: {f1_bayesian:.2f}')

Average Rating - Precision: 0.00, Recall: 0.00, F1-Score: 0.00
Weighted Scoring - Precision: 0.40, Recall: 0.20, F1-Score: 0.27
Bayesian Scoring - Precision: 0.00, Recall: 0.00, F1-Score: 0.00
